<a href="https://colab.research.google.com/github/Anjasfedo/pytorch/blob/main/01_pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Workflow

PyTorch end-to-end workflow

In [2]:
what_were_covering = {1: "data (prepare & load)",
           2: "build model",
           3: "fitting model to data (training)",
           4: "make predict and evaluate model (inference)",
           5: "save & load model",
           6: "put all together"}

In [1]:
import torch
from torch import nn # nn contain all PyTorch building block for neural network
import matplotlib.pyplot as plt

torch.__version__

'2.3.0+cu121'

## Data preparing & Loading

Data can be almost anything on machine learning
- Excel spreadsheet
- Text
- Images
- Videos
- Audio
- DNA

machine learning is game of two part
1. get data into numerical representation
2. build model to learn patern based on that numerical representation

to showcase this, lets create some *known* data from linear regression formula

use linear regression formula to make straight line with known *parameters*



In [8]:
# create known parameter

weight = 0.7
bias = 0.3

# create
start = 0
end = 1
step = 0.02

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [6]:
len(X), len(y)

(50, 50)